In [1]:
import pandas as pd

dir_path = "workflow/data"

files = [
    "FreightRates", 
    "OrderList", 
    "PlantPorts", 
    "ProductsPerPlant", 
    "VmiCustomers", 
    "WhCapacities", 
    "WhCosts"
]

files = {file: pd.read_csv(f"{dir_path}/{file}.csv", index_col=0) for file in files}

for k, v in files.items():
    files[k].columns = [c.strip().replace(" ", "_").lower() for c in files[k].columns]
    v_copy = v.copy()
    v_copy.index.name = k
    display(v_copy.head(2))

,carrier,orig_port_cd,dest_port_cd,minm_wgh_qty,max_wgh_qty,svc_cd,minimum_cost,rate,mode_dsc,tpt_day_cnt,carrier_type
FreightRates,,,,,,,,,,,
0,V444_6,PORT08,PORT09,250.0,499.99,DTD,43.2272,0.7132,AIR,2,V88888888_0
1,V444_6,PORT08,PORT09,65.0,69.99,DTD,43.2272,0.7512,AIR,2,V88888888_0


,order_id,order_date,origin_port,carrier,tpt,service_level,ship_ahead_day_count,ship_late_day_count,customer,product_id,plant_code,destination_port,unit_quantity,weight
OrderList,,,,,,,,,,,,,,
0,1.447296e+09,2013-05-26,PORT09,V44_3,1,CRF,3,0,V55555_53,1700106,PLANT16,PORT09,808,14.30
1,1.447158e+09,2013-05-26,PORT09,V44_3,1,CRF,3,0,V55555_53,1700106,PLANT16,PORT09,3188,87.94


,plant_code,port
PlantPorts,,
0,PLANT01,PORT01
1,PLANT01,PORT02


,plant_code,product_id
ProductsPerPlant,,
0,PLANT15,1698815
1,PLANT17,1664419


,plant_code,customers
VmiCustomers,,
0,PLANT02,V5555555555555_16
1,PLANT02,V555555555555555_29


,plant_id,daily_capacity
WhCapacities,,
0,PLANT15,11
1,PLANT17,8


,wh,cost/unit
WhCosts,,
0,PLANT15,1.415063
1,PLANT17,0.428947


In [2]:
df0 = files["OrderList"][["order_id", "product_id", "origin_port", "destination_port", "unit_quantity", "weight"]].copy()
df0["order_id"] = df0["order_id"].astype(int)

df0.head()

,order_id,product_id,origin_port,destination_port,unit_quantity,weight
0,1447296446,1700106,PORT09,PORT09,808,14.30
1,1447158014,1700106,PORT09,PORT09,3188,87.94
2,1447138898,1700106,PORT09,PORT09,2331,61.20
3,1447363527,1700106,PORT09,PORT09,847,16.16
4,1447363980,1700106,PORT09,PORT09,2163,52.34


In [3]:
files["ProductsPerPlant"].head()

,plant_code,product_id
0,PLANT15,1698815
1,PLANT17,1664419
2,PLANT17,1664426
3,PLANT17,1672826
4,PLANT17,1674916


In [4]:
df1 = pd.merge(df0, files["ProductsPerPlant"], on="product_id")
df1.rename(columns={"plant_code": "plant_id"}, inplace=True)

df1.head()

,order_id,product_id,origin_port,destination_port,unit_quantity,weight,plant_id
0,1447296446,1700106,PORT09,PORT09,808,14.30,PLANT16
1,1447158014,1700106,PORT09,PORT09,3188,87.94,PLANT16
2,1447138898,1700106,PORT09,PORT09,2331,61.20,PLANT16
3,1447363527,1700106,PORT09,PORT09,847,16.16,PLANT16
4,1447363980,1700106,PORT09,PORT09,2163,52.34,PLANT16


In [5]:
files["WhCapacities"].head()

,plant_id,daily_capacity
0,PLANT15,11
1,PLANT17,8
2,PLANT18,111
3,PLANT05,385
4,PLANT02,138


In [6]:
df2 = pd.merge(df1, files["WhCapacities"], on="plant_id")
df2.rename(columns={"daily_capacity": "plant_capacity"}, inplace=True)

df2.head()

,order_id,product_id,origin_port,destination_port,unit_quantity,weight,plant_id,plant_capacity
0,1447296446,1700106,PORT09,PORT09,808,14.30,PLANT16,457
1,1447158014,1700106,PORT09,PORT09,3188,87.94,PLANT16,457
2,1447138898,1700106,PORT09,PORT09,2331,61.20,PLANT16,457
3,1447363527,1700106,PORT09,PORT09,847,16.16,PLANT16,457
4,1447363980,1700106,PORT09,PORT09,2163,52.34,PLANT16,457


In [10]:
files["WhCosts"].head()

,wh,cost/unit
0,PLANT15,1.415063
1,PLANT17,0.428947
2,PLANT18,2.036254
3,PLANT05,0.488144
4,PLANT02,0.477504


In [12]:
costs = files["WhCosts"].copy()
costs.rename(columns={"wh": "plant_id", "cost/unit": "unit_storage_cost"}, inplace=True)

df3 = pd.merge(df2, costs, on="plant_id")

df3["total_storage_cost"] = df3["unit_storage_cost"] * df3["unit_quantity"]

df3.head()

,order_id,product_id,origin_port,destination_port,unit_quantity,weight,plant_id,plant_capacity,unit_storage_cost,total_storage_cost
0,1447296446,1700106,PORT09,PORT09,808,14.30,PLANT16,457,1.919808,1551.204466
1,1447158014,1700106,PORT09,PORT09,3188,87.94,PLANT16,457,1.919808,6120.346335
2,1447138898,1700106,PORT09,PORT09,2331,61.20,PLANT16,457,1.919808,4475.071301
3,1447363527,1700106,PORT09,PORT09,847,16.16,PLANT16,457,1.919808,1626.076959
4,1447363980,1700106,PORT09,PORT09,2163,52.34,PLANT16,457,1.919808,4152.543639


In [16]:
plant_ports = files["PlantPorts"].copy()
plant_ports.rename(columns={"plant_code": "plant_id", "port": "plant_port"}, inplace=True)

df4 = pd.merge(df3, plant_ports, on="plant_id")

df4.head()

,order_id,product_id,origin_port,destination_port,unit_quantity,weight,plant_id,plant_capacity,unit_storage_cost,total_storage_cost,plant_port
0,1447296446,1700106,PORT09,PORT09,808,14.30,PLANT16,457,1.919808,1551.204466,PORT09
1,1447158014,1700106,PORT09,PORT09,3188,87.94,PLANT16,457,1.919808,6120.346335,PORT09
2,1447138898,1700106,PORT09,PORT09,2331,61.20,PLANT16,457,1.919808,4475.071301,PORT09
3,1447363527,1700106,PORT09,PORT09,847,16.16,PLANT16,457,1.919808,1626.076959,PORT09
4,1447363980,1700106,PORT09,PORT09,2163,52.34,PLANT16,457,1.919808,4152.543639,PORT09


In [24]:
files["FreightRates"][files["FreightRates"]["orig_port_cd"] == "PORT09"]

,carrier,orig_port_cd,dest_port_cd,minm_wgh_qty,max_wgh_qty,svc_cd,minimum_cost,rate,mode_dsc,tpt_day_cnt,carrier_type
38,V444_8,PORT09,PORT09,500.0,1999.99,DTD,17.6888,0.0332,AIR,14,V88888888_0
39,V444_8,PORT09,PORT09,2000.0,99999.99,DTD,17.6888,0.0332,AIR,14,V88888888_0
40,V444_8,PORT09,PORT09,0.0,99.99,DTD,17.6888,0.0332,AIR,14,V88888888_0
41,V444_8,PORT09,PORT09,250.0,499.99,DTD,17.6888,0.0332,AIR,14,V88888888_0
42,V444_8,PORT09,PORT09,100.0,249.99,DTD,17.6888,0.0332,AIR,14,V88888888_0
88,V444_8,PORT09,PORT09,2000.0,99999.99,DTD,23.5400,0.0476,AIR,5,V88888888_0
89,V444_8,PORT09,PORT09,500.0,1999.99,DTD,23.5400,0.0476,AIR,5,V88888888_0
90,V444_8,PORT09,PORT09,100.0,249.99,DTD,23.5400,0.0476,AIR,5,V88888888_0
91,V444_8,PORT09,PORT09,0.0,99.99,DTD,23.5400,0.0476,AIR,5,V88888888_0
92,V444_8,PORT09,PORT09,250.0,499.99,DTD,23.5400,0.0476,AIR,5,V88888888_0


In [19]:
files["VmiCustomers"]

,plant_code,customers
0,PLANT02,V5555555555555_16
1,PLANT02,V555555555555555_29
2,PLANT02,V555555555_3
3,PLANT02,V55555555555555_8
4,PLANT02,V55555555_9
5,PLANT02,V55555_10
6,PLANT02,V55555555_5
7,PLANT06,V555555555555555_18
8,PLANT06,V55555_10
9,PLANT10,V555555555555555_29
